In [124]:
import os
import pandas as pd
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
from datetime import timedelta
cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']


dict = {}
for cantonId in cantonKeys:
    dict[cantonId] = pd.read_csv("data/dailyFeatures/"+cantonId+".csv").set_index('date')
    dict[cantonId].index = pd.to_datetime(dict[cantonId].index)




In [115]:
# generate date ranges
dataStart = pd.Timestamp('2020-02-15')
dataEnd = pd.Timestamp('2021-04-05')

daysIn = 7 # last n days of input
daysOut = 7 # next n days of output

listOfInputIntervals = []
listOfOutputIntervals = []
for e in pd.date_range(start=dataStart,end=dataEnd, freq='D'):
    if (e+timedelta(days = daysIn+daysOut-1) <= dataEnd.date()):
        listOfInputIntervals.append((e.date(),(e+timedelta(days = (daysIn-1))).date()))
        listOfOutputIntervals.append(((e+timedelta(days = daysIn)).date(),(e+timedelta(days = daysIn+daysOut-1)).date()))
    
#display(listOfInputIntervals)
#display(listOfOutputIntervals)

In [117]:
######### constructing the inputs #########
for cantonId in cantonKeys:
    display(cantonId)
    features = pd.DataFrame()
    
    # construction of input features
    for t in listOfInputIntervals:
        # features which will be averaged over the whole input interval
        averageFeatures = ['Cases inz_entries 0 - 9','Cases inz_entries 10 - 19','Cases inz_entries 20 - 29',
                  'Cases inz_entries 30 - 39','Cases inz_entries 40 - 49','Cases inz_entries 50 - 59',
                  'Cases inz_entries 60 - 69','Cases inz_entries 70 - 79','Cases inz_entries 80+',
                  'Cases inzsumTotal 0 - 9','Cases inzsumTotal 10 - 19','Cases inzsumTotal 20 - 29',
                  'Cases inzsumTotal 30 - 39','Cases inzsumTotal 40 - 49','Cases inzsumTotal 50 - 59',
                  'Cases inzsumTotal 60 - 69','Cases inzsumTotal 70 - 79','Cases inzsumTotal 80+',
                  'Death inz_entries 0 - 9','Death inz_entries 10 - 19','Death inz_entries 20 - 29',
                  'Death inz_entries 30 - 39','Death inz_entries 40 - 49','Death inz_entries 50 - 59',
                  'Death inz_entries 60 - 69','Death inz_entries 70 - 79','Death inz_entries 80+',
                  'Death inzsumTotal 0 - 9','Death inzsumTotal 10 - 19','Death inzsumTotal 20 - 29',
                  'Death inzsumTotal 30 - 39','Death inzsumTotal 40 - 49','Death inzsumTotal 50 - 59',
                  'Death inzsumTotal 60 - 69','Death inzsumTotal 70 - 79','Death inzsumTotal 80+',
                  'Hosp inz_entries 0 - 9','Hosp inz_entries 10 - 19','Hosp inz_entries 20 - 29',
                  'Hosp inz_entries 30 - 39','Hosp inz_entries 40 - 49','Hosp inz_entries 50 - 59',
                  'Hosp inz_entries 60 - 69','Hosp inz_entries 70 - 79','Hosp inz_entries 80+',
                  'Hosp inzsumTotal 0 - 9','Hosp inzsumTotal 10 - 19','Hosp inzsumTotal 20 - 29',
                  'Hosp inzsumTotal 30 - 39','Hosp inzsumTotal 40 - 49','Hosp inzsumTotal 50 - 59',
                  'Hosp inzsumTotal 60 - 69','Hosp inzsumTotal 70 - 79','Hosp inzsumTotal 80+',
                  'Cases inz_entries female','Cases inz_entries male','Cases inzsumTotal female',
                  'Cases inzsumTotal male','Death inz_entries female','Death inz_entries male',
                  'Death inzsumTotal female','Death inzsumTotal male','Hosp inz_entries female',
                  'Hosp inz_entries male','Hosp inzsumTotal female','Hosp inzsumTotal male', 
                  'VaccDosesAdministered per100PersonsTotal',
                  'FullyVaccPersons per100PersonsTotal',
                  'anteil_pos',
                  'variant_error',
                  'case_inzsumTotal','hosp_inzsumTotal','death_inzsumTotal','test_inzsumTotal','case_inz_entries',
                  'hosp_inz_entries','death_inz_entries','test_inz_entries','testPositvity',
                  'median_R_mean','R_error',
                  'meanNeighborIncidence','maxNeighborIncidence',
                  'kofStrigency',
                  'Borders','Events','Gatherings/private events','Demonstrations',
                  'Primary (includes kindergarten) and lower secondary school','Upper secondary school, vocational schools and higher education',
                  'universities and other educational establishments\xa0','Mountain railways','Homeworking','Restaurants',
                  'Discos/Nightclubs','Shops/Markets','Penalties','Cultural, entertainment and recreational facilities',
                  'Sport/Wellness facilities','Sport activities','Religious services','Singing allowed','maskMandatories',
                  'ICU_AllPatients_inz','ICU_Covid19Patients_inz','ICU_Capacity_inz','Total_AllPatients_inz',
                  'Total_Covid19Patients_inz','Total_Capacity_inz','ICU_NonCovid19Patients_inz','ICU_FreeCapacity_inz',
                  'Total_NonCovid19Patients_inz','Total_FreeCapacity_inz'
                 ]

        # features which will be direct input for every day of the input interval
        # attention: this can potentially increase the number of input features significantly
        # added features are len(directFea)*daysIn
        # only add features for which have a large variance from one day to another day
        directFeatures = ['retail_and_recreation_percent_change_from_baseline',
                     'grocery_and_pharmacy_percent_change_from_baseline',
                     'parks_percent_change_from_baseline',
                     'transit_stations_percent_change_from_baseline',
                     'workplaces_percent_change_from_baseline',
                     'residential_percent_change_from_baseline',
                     'intervistaMob',
                     'isHolyday',
                     'temp_min','temp_max','clouds','precipitation']

        # average features
        featureRow = dict[cantonId][t[0]:t[1]][averageFeatures].mean().to_frame().transpose()

        # direct features
        for f in directFeatures:
            directFe = dict[cantonId][t[0]:t[1]][[f]].transpose()
            directFe.columns = [f+'_day_'+str(d) for d in range(0,daysIn)]
            directFe = directFe.reset_index().drop(['index'], axis=1)
            featureRow = pd.concat([featureRow,directFe], axis=1)
        
        
        # future features
        futureFeatures = ['temp_min','temp_max','clouds','precipitation']
        indexOfInputTuple = listOfInputIntervals.index(t)
        ot = listOfOutputIntervals[indexOfInputTuple]
        for ff in futureFeatures:
            futureFe = dict[cantonId][ot[0]:ot[1]][[ff]].transpose()
            futureFe.columns = [ff+'_future_day_'+str(d) for d in range(0,daysOut)]
            futureFe = futureFe.reset_index().drop(['index'], axis=1)
            featureRow = pd.concat([featureRow,futureFe], axis=1)
        
        features = features.append(featureRow, ignore_index=True)

    # static cantonal features
    staticCantonal = pd.read_excel("static_data/staticCantonalData.xlsx").set_index('canton').transpose()
    # households
    households = ['1PersonHouseholds', '2PersonHouseholds','3PersonHouseholds', '4PersonHouseholds', 
                  '5PersonHouseholds','6+PersonHouseholds']
    for h in households:
        features[[h+"_perc"]] = staticCantonal.loc[[cantonId]][h][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    features[['averageHousehold']] = staticCantonal.loc[[cantonId]]['residents'][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    # add static features
    staticFeatures = ['percentage 65 years or over','urbanPopulationPercent','homeownershipPercent', 
                      'livingSpaceInm2','carsPer1000inhabitants', 'publicTransportationPercent',
                      'privateMotorisedTransportPercent','DoctorsPer100Kinhabitants','residentsPerKm2']
    for f in staticFeatures:
        features[[f]] = staticCantonal.loc[[cantonId]][f][0]
    # construct settlement area feature
    residents = staticCantonal.loc[[cantonId]]['residents'][0]
    settlementArea = staticCantonal.loc[[cantonId]]['areaInKm2'][0]*(staticCantonal.loc[[cantonId]]['settlementAreaPercent'][0]/100)
    features[['residentsPerKm2SettlementArea']] = residents/settlementArea    
    
    if not os.path.exists('data/features'):
        os.makedirs('data/features')
    features.to_csv('data/features/'+cantonId+'.csv', index=False)
    #display(features)

'AG'

'AI'

'AR'

'BE'

'BL'

...

'UR'

'VD'

'VS'

'ZG'

'ZH'

In [118]:
######### constructing the ouputs #########
for cantonId in cantonKeys:
    outputs = pd.DataFrame()
    for t in listOfOutputIntervals:
        outputCols = ['hosp_inz_entries','death_inz_entries','testPositvity','googleMobility']

        outputRow = dict[cantonId][t[0]:t[1]][outputCols].mean().to_frame().transpose()

        '''
        outputRow = pd.DataFrame()
        # direct features
        for o in outputCols:
            directOu = dict[cantonId][t[0]:t[1]][[o]].transpose()
            directOu.columns = [o+'_day_'+str(d) for d in range(0,daysOut)]
            directOu = directOu.reset_index().drop(['index'], axis=1)
            outputRow = pd.concat([outputRow,directOu], axis=1)
        '''    
        outputs = outputs.append(outputRow, ignore_index=True)
    
    if not os.path.exists('data/outputs'):
        os.makedirs('data/outputs')
    outputs.to_csv('data/outputs/'+cantonId+'.csv', index=False)
#display(outputs)




In [119]:
# test cantons are (large, medium, small) (de, fr, de) SG,NE,NW

trainCantons = [canton for canton in cantonKeys if canton not in ['SG','NE','NW']]
# construct train set
train_features = pd.DataFrame()
train_labels = pd.DataFrame()
for cantonId in trainCantons:
    train_features = train_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    train_labels = train_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))

test_features = pd.DataFrame()
test_labels = pd.DataFrame()
# construct test set
for cantonId in ['SG','NE','NW']:
    test_features = test_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    test_labels = test_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))

train_features.to_csv('train_features.csv', index=False)
train_labels.to_csv('train_labels.csv', index=False)
test_features.to_csv('test_features.csv', index=False)
test_labels.to_csv('test_labels.csv', index=False)

In [120]:
# output log file
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
import lightgbm as lightgbm
import xgboost as xgb



start = time.time()
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')
test_labels = pd.read_csv('test_labels.csv')



#pip = Pipeline([('std_scaler', StandardScaler())])
pip = Pipeline([('minmax_scaler', MinMaxScaler())])
featureNames = train_features.columns
train_features = pip.fit_transform(train_features)
cv = 5

display(train_labels)

,hosp_inz_entries,death_inz_entries,testPositvity,googleMobility
0,0.021429,0.000000,0.014211,5.690476
1,0.062857,0.000000,0.032002,3.857143
2,0.062857,0.000000,0.044182,3.714286
3,0.104286,0.000000,0.048949,1.619048
4,0.104286,0.000000,0.037903,2.142857
...,...,...,...,...
9264,0.768571,0.090000,0.046355,-22.166667
9265,0.667143,0.098571,0.046011,-22.166667
9266,0.611429,0.071429,0.045847,-22.166667
9267,0.564286,0.080000,0.044938,-22.166667


In [121]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # SVR
    display("---SVR---"+label+"---")
    start = time.time()
    parameters = {'C':[0.1, 1]}
    reg = GridSearchCV(svm.SVR(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---SVR---hosp_inz_entries---'

SVR(C=1)

'Scores:[[0.05179367 0.06186199 0.0535296  0.08717979 0.0392781 ]]'

'Mean:0.05872863042397854'

'Standard deviation:0.015954520152582564'

...

SVR(C=1)

'Scores:[[0.05769632 0.06602492 0.04642345 0.04519649 0.04946118]]'

'Mean:0.0529604727255202'

'Standard deviation:0.007853217045760596'

'----------End of evaluating (2.3682315349578857)----------'

In [95]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # Ridge regression
    display("---Ridge---"+label+"---")
    start = time.time()
    parameters = {'alpha':[0.1, 1, 10]}
    reg = GridSearchCV(linear_model.Ridge(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    #display(pd.DataFrame(reg.best_estimator_.coef_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---Ridge---hosp_inz_entries---'

Ridge(alpha=0.1)

'Scores:[[0.04224647 0.05224684 0.03448905 0.0446484  0.03428489]]'

'Mean:0.04158312922288176'

'Standard deviation:0.006740042140557605'

...

Ridge(alpha=1)

'Scores:[[0.03543829 0.03733242 0.03328197 0.03058123 0.03537479]]'

'Mean:0.0344017411249536'

'Standard deviation:0.0023004492708929864'

'----------End of evaluating (0.608684778213501)----------'

In [122]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # Lasso regression
    display("---Lasso---"+label+"---")
    start = time.time()
    parameters = {'alpha':[0.00001, 0.0001]}
    reg = GridSearchCV(linear_model.Lasso(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.coef_.transpose(), index=featureNames, columns=['coefficients']).sort_values(['coefficients'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---Lasso---hosp_inz_entries---'

Lasso(alpha=0.0001)

'Scores:[[0.02664086 0.02515486 0.02356835 0.02719636 0.01745393]]'

'Mean:0.024002871285291914'

'Standard deviation:0.0035088985543820232'

...

'Scores:[[0.0292514  0.03101672 0.02171582 0.02032745 0.02530927]]'

'Mean:0.025524133438071077'

'Standard deviation:0.004138019072258814'

,coefficients
Death inz_entries female,0.507942
Death inz_entries male,0.502378
Cases inz_entries 80+,0.082079
Death inz_entries 80+,0.073473
hosp_inz_entries,0.036926
...,...
clouds_day_0,-0.001823
Cases inzsumTotal 80+,-0.002358
Death inzsumTotal 80+,-0.006293
maxNeighborIncidence,-0.006640


'----------End of evaluating (5.260971307754517)----------'

In [106]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # MLP regression
    display("---MLP---"+label+"---")
    start = time.time()
    parameters = {'hidden_layer_sizes':[(100,50,50,20,10,10)]}
    reg = GridSearchCV(MLPRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---MLP---hosp_inz_entries---'

MLPRegressor(hidden_layer_sizes=(100, 50, 50, 20, 10, 10))

'Scores:[[0.04622179 0.04572822 0.04308994 0.07297331 0.03196155]]'

'Mean:0.04799496317317258'

'Standard deviation:0.013515397105826426'

...

MLPRegressor(hidden_layer_sizes=(100, 50, 50, 20, 10, 10))

'Scores:[[0.04133558 0.05863064 0.04531868 0.0450905  0.04821174]]'

'Mean:0.04771742845651561'

'Standard deviation:0.005877503221681948'

'----------End of evaluating (7.370590686798096)----------'

In [101]:
for label in ['death_inz_entries','hosp_inz_entries', 'testPositvity','googleMobility']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()

    # RandomForestRegressor
    display("---RandomForestRegressor---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[20]}
    reg = GridSearchCV(RandomForestRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))


'---RandomForestRegressor---death_inz_entries---'

RandomForestRegressor(n_estimators=20)

'Scores:[[0.04198309 0.05013015 0.03943916 0.03875326 0.04255276]]'

'Mean:0.04257168297808148'

'Standard deviation:0.0040464871432413785'

...

'Scores:[[0.04420332 0.09632913 0.06832748 0.07425238 0.06026043]]'

'Mean:0.06867454750119298'

'Standard deviation:0.017121020677670677'

,importance
transit_stations_percent_change_from_baseline_day_13,5.563682e-01
parks_percent_change_from_baseline_day_13,7.471478e-02
test_inzsumTotal,2.311992e-02
parks_percent_change_from_baseline_day_12,2.311731e-02
retail_and_recreation_percent_change_from_baseline_day_13,2.232816e-02
...,...
Death inz_entries 0 - 9,9.875664e-08
VaccDosesAdministered per100PersonsTotal,6.829026e-09
FullyVaccPersons per100PersonsTotal,6.027901e-09
Death inz_entries 10 - 19,3.523223e-19


'----------End of evaluating (51.5964035987854)----------'

In [125]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()
    
    # lightGBM
    display("---lightGBM---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[100]}
    reg = GridSearchCV(lightgbm.LGBMRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    display(reg.best_estimator_)
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---lightGBM---hosp_inz_entries---'

LGBMRegressor()

'Scores:[[0.03362329 0.03104012 0.02767795 0.05858911 0.02081361]]'

'Mean:0.03434881511942382'

'Standard deviation:0.01285898972228739'

...

'Scores:[[0.03476192 0.0384098  0.02621302 0.02955132 0.03107556]]'

'Mean:0.032002324022578535'

'Standard deviation:0.004220271230922381'

,importance
death_inz_entries,293
Death inz_entries female,194
Death inz_entries male,194
Cases inz_entries 80+,104
Death inz_entries 80+,64
Death inz_entries 60 - 69,50
hosp_inz_entries,47
Hosp inz_entries 60 - 69,45
Hosp inz_entries 80+,44
Death inz_entries 70 - 79,43


'----------End of evaluating (4.198756694793701)----------'

In [126]:
for label in ['hosp_inz_entries', 'testPositvity','googleMobility','death_inz_entries']:
    tl = pip.fit_transform(train_labels[[label]]).ravel()
    
    # XGBoost
    display("---XGBoost---"+label+"---")
    start = time.time()
    parameters = {'n_estimators':[10], 'reg_alpha': [0], 'reg_lambda': [1]}
    reg = GridSearchCV(xgb.XGBRegressor(), parameters, cv=cv, n_jobs=-1, scoring='neg_mean_squared_error')
    reg.fit(train_features, tl)
    #display(pd.DataFrame(reg.cv_results_))
    #display(reg.best_estimator_.get_xgb_params())
    scores = pd.DataFrame(reg.cv_results_).loc[[reg.best_index_]][['split'+str(i)+'_test_score' for i in range(0,cv)]].values
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    display("Mean:"+str(scores.mean()))
    display("Standard deviation:"+str(scores.std()))
    display(pd.DataFrame(reg.best_estimator_.feature_importances_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
    display("----------End of evaluating (%s)----------" % (time.time() - start))

'---XGBoost---hosp_inz_entries---'

'Scores:[[0.03945924 0.04106757 0.03549687 0.06657774 0.02724659]]'

'Mean:0.04196960412206989'

'Standard deviation:0.013201055874427471'

,importance
Hosp inz_entries male,0.306881
Hosp inz_entries female,0.113868
Events,0.065002
Hosp inz_entries 30 - 39,0.045581
"Cultural, entertainment and recreational facilities",0.040340
Discos/Nightclubs,0.024777
Gatherings/private events,0.022566
Death inz_entries 80+,0.019448
kofStrigency,0.013878
residential_percent_change_from_baseline_day_0,0.012585


...

'Scores:[[0.03892503 0.04773417 0.03460766 0.0378127  0.03688034]]'

'Mean:0.03919197885881143'

'Standard deviation:0.004501162281226967'

,importance
Death inz_entries male,0.246495
Death inz_entries 80+,0.157669
Cases inz_entries female,0.079390
Hosp inzsumTotal 50 - 59,0.066965
Hosp inzsumTotal 80+,0.031209
Death inzsumTotal female,0.021710
Death inz_entries female,0.020479
Cases inz_entries 80+,0.019502
privateMotorisedTransportPercent,0.019168
Hosp inz_entries female,0.017933


'----------End of evaluating (35.6097629070282)----------'

In [139]:
# PCA
from sklearn.decomposition import IncrementalPCA

ipca = IncrementalPCA()
ipca.fit(train_features)
display(pd.DataFrame(ipca.singular_values_.transpose(), index=featureNames, columns=['singular value']).sort_values(['singular value'], ascending=False))

,singular value
Cases inz_entries 0 - 9,1.683747e+02
Cases inz_entries 10 - 19,1.054882e+02
Cases inz_entries 20 - 29,8.109005e+01
Cases inz_entries 30 - 39,7.285874e+01
Cases inz_entries 40 - 49,6.620652e+01
Cases inz_entries 50 - 59,6.239412e+01
Cases inz_entries 60 - 69,5.786845e+01
Cases inz_entries 70 - 79,5.081096e+01
Cases inz_entries 80+,4.907019e+01
Cases inzsumTotal 0 - 9,4.838226e+01
